In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
print('Libraries installed!')

data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
can_data = BeautifulSoup(data, 'html.parser')

postalCodeList = []
boroughList = []
neighborhoodList = []

for row in can_data.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # remove the new line char from neighborhood cell
        
tor_neighorhood = [('PostalCode', postalCodeList),
                      ('Borough', boroughList),
                      ('Neighborhood', neighborhoodList)]
df = pd.DataFrame.from_dict(dict(tor_neighorhood))
# df

df_drop = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_drop

df_grp = df_drop.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
# df_grp

neigh_row = df_grp.Neighborhood == 'Not assigned'
df_grp.loc[neigh_row, 'Neighborhood'] = df_grp.loc[neigh_row, 'Borough']
# df_grp[neigh_row]


df_result = df_grp
df_result.shape


Libraries installed!


(103, 3)

In [18]:


# Part 2: Getting coordinates and add to the Toronto DataFrame

!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coors = pd.read_csv('toronto_coordinates.csv')


# print(coors.shape)
coors.head()

Coordinates downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
toronto_df_temp = df_result.set_index('PostalCode')
coors_temp = coors.set_index('Postal Code')
toronto_df_coors = pd.concat([toronto_df_temp, coors_temp], axis=1, join='inner')

toronto_df_coors.index.name = 'PostalCode'
toronto_df_coors.reset_index(inplace=True)

print(toronto_df_coors.shape)
toronto_df_coors.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
